In [1]:
import re
from benchmarking import *

In [2]:
cache = {}
def shootme(seed):
    if seed not in cache:
        ret = run_shootme_seed(seed)
        cache[seed] = ret
    return cache[seed]

In [3]:
def summary(ret):
    print('\n'.join(ret.splitlines()[:16]))

In [4]:
summary(shootme(12))

 0: 
 1: 8H 
 2: 6C -3D
 3: JS -JD-5S
 4: 9S -7C-KD-2H
 5: AC -2C-JC-8D-QH
 6: 5C -9D-3S-2S-6S-4S
 7: JH -9H-TS-4D-5D-6D-5H
 8: 3H TD 2D AS 4H 6H AH 8S TH QD KH QS AD KS 7D 4C TC 7H 7S 3C 8C QC 9C KC 
 9: 
10: 
11: 
12: 
Minimum Moves Needed: 86

Minimal solution in 116 moves. Took 5221604 ms.


In [5]:
# got these results by trying seeds in the past
impossible_seeds = [5, 8, 11]
indeterminate_seeds = []

In [6]:
sorted(cache.keys())

[12]

In [7]:
impossible = shootme(5)
winnable = shootme(1)

In [8]:
took_time = re.compile(r'.* Took (\d+) ms.$')
win_result = re.compile(r'Minimal solution in (\d+) moves. Took (\d+) ms.')

In [9]:
print('\n'.join(winnable.splitlines()[:13]))

 0: 
 1: 4D 
 2: 4C -3C
 3: 6H -7D-9S
 4: JD -2C-7S-AH
 5: AS -TS-2H-6S-AD
 6: 5C -JC-KS-2D-8H-4H
 7: KD -6C-JH-5D-3S-4S-3H
 8: 9D TC 2S QC KH 5S 9C AC QH QD 8S 6D 7C 9H JS QS 8C 5H TD 3D KC 7H 8D TH 
 9: 
10: 
11: 
12: 


In [10]:
def parse_winnable(ret):
    lines = ret.splitlines()
    solution_result = lines[15]
    moveseq = lines[-2]
    deck = '\n'.join(lines[:13])
    move_count, ms = map(int, win_result.match(solution_result).groups())
    return {
        'solved': True,
        'impossible': False,
        'move_count': move_count,
        'time_ms': ms,
        'result': solution_result,
        'moves': moveseq,
        'deck': deck,
    }

parse_winnable(winnable)

{'solved': True,
 'impossible': False,
 'move_count': 117,
 'time_ms': 9115068,
 'result': 'Minimal solution in 117 moves. Took 9115068 ms.',
 'moves': '5S F5 DR1 WS 16 71 F7 DR5 W7 27 F2 63-2 F6 23 DR1 W2 W7 W6 DR1 NEW DR1 W1 DR2 W2 62-2 F6 W6 WC 54 F5 41-2 F4 4C F4 53 F5 34-5 F3 53 F5 5D 65-2 F6 6D F6 7D 46-6 F4 4H 6H 6C 7C 6D 6C 73 7C F7 W2 W6 W4 DR1 WC 62-4 F6 DR1 W4 W5 74 F7 7D F7 7S F7 7S F7 7H 6H 2S 3H 2H 3S 2S W1 WD 3D F3 DR1 WD WH 2H WC WS 1H 2C 3S DR1 WH WC WD 1S 2D 1D 2C 4H 5S 1C 2D 4S 5H 1D 2C 4H 5S ',
 'deck': ' 0: \n 1: 4D \n 2: 4C -3C\n 3: 6H -7D-9S\n 4: JD -2C-7S-AH\n 5: AS -TS-2H-6S-AD\n 6: 5C -JC-KS-2D-8H-4H\n 7: KD -6C-JH-5D-3S-4S-3H\n 8: 9D TC 2S QC KH 5S 9C AC QH QD 8S 6D 7C 9H JS QS 8C 5H TD 3D KC 7H 8D TH \n 9: \n10: \n11: \n12: '}

In [11]:
def parse_impossible(ret):
    lines = ret.splitlines()
    solution_result = lines[15]
    moveseq = lines[-2]
    deck = '\n'.join(lines[:13])
    ms = int(took_time.match(solution_result).groups()[0])
    return {
        'solved': False,
        'impossible': True,
        'time_ms': ms,
        'result': solution_result,
        'deck': deck,
    }

parse_impossible(impossible)

{'solved': False,
 'impossible': True,
 'time_ms': 225038,
 'result': 'Impossible. Max cards in foundation 6 at 40 moves. Took 225038 ms.',
 'deck': ' 0: \n 1: 3D \n 2: 9C -JH\n 3: KS -9S-7D\n 4: KC -6C-9H-5S\n 5: 4D -QD-7S-4S-3C\n 6: JD -5D-5H-AD-KH-4H\n 7: 6D -AS-2D-2C-TC-5C-TH\n 8: 4C QS 7H 3S TS 9D 8H JS 8D AC 7C 2H QC TD 6H AH JC 6S QH KD 2S 8C 3H 8S \n 9: \n10: \n11: \n12: '}